In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import ollama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [2]:
doc_path = r"C:\ml_algorithms\LangGraphs_n_langchain\langchain\data\the-psychology-of-money-by-morgan-housel.pdf"

In [3]:
model = "gemma3:12b"

In [4]:
if doc_path:
    loader = PyPDFLoader(doc_path)
    documents = loader.load()
    print("done loading...")
else:
    print("Please upload a file the mentioned file doesn't exist!!")

done loading...


In [5]:
len(documents)

253

In [6]:
content = documents[0].page_content
print(content[0:500])

In [7]:
# split and chunks
text_spliter = RecursiveCharacterTextSplitter(chunk_size = 1200, chunk_overlap = 400)
chunks = text_spliter.split_documents(documents)
print("done splitting...")
print(f"Total chunks: {len(chunks)}")

done splitting...
Total chunks: 412


In [8]:
chunks[15].page_content

'The money industry—investing, personal ﬁnance, business planning—is another story.\nFinance has scooped up the smartest minds coming from top universities over the last twodecades. Financial Engineering was the most popular major in Princeton’s School ofEngineering a decade ago. Is there any evidence it has made us better investors?\nI have seen none.\nThrough collective trial and error over the years we learned how to become better farmers,skilled plumbers, and advanced chemists. But has trial and error taught us to becomebetter with our personal ﬁnances? Are we less likely to bury ourselves in debt? More likelyto save for a rainy day? Prepare for retirement? Have realistic views about what moneydoes, and doesn’t do, to our happiness?\nI’ve seen no compelling evidence.\nMost of the reason why, I believe, is that we think about and are taught about money inways that are too much like physics (with rules and laws) and not enough like psychology(with emotions and nuance).\nAnd that, to 

In [9]:
chunks[16].page_content

'I’ve seen no compelling evidence.\nMost of the reason why, I believe, is that we think about and are taught about money inways that are too much like physics (with rules and laws) and not enough like psychology(with emotions and nuance).\nAnd that, to me, is as fascinating as it is important.\nMoney is everywhere, it aﬀects all of us, and confuses most of us. Everyone thinks about it alittle diﬀerently. It oﬀers lessons on things that apply to many areas of life, like risk,conﬁdence, and happiness. Few topics oﬀer a more powerful magnifying glass that helpsexplain why people behave the way they do than money. It is one of the greatest shows onEarth.\nMy own appreciation for the psychology of money is shaped by more than a decade ofwriting on the topic. I began writing about ﬁnance in early 2008. It was the dawn of aﬁnancial crisis and the worst recession in 80 years.\nTo write about what was happening, I wanted to ﬁgure out what was happening. But theﬁrst thing I learned after the ﬁna

In [10]:
ollama.pull("nomic-embed-text:v1.5")

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [11]:
# Save to disk
persist_dir = "./chroma_db"

In [12]:
# # Create and persist vector db
# vector_db = Chroma.from_documents(
#     documents=chunks,
#     embedding=OllamaEmbeddings(model="nomic-embed-text:v1.5"),
#     collection_name="psychology_of_money_rag",
#     persist_directory=persist_dir
# )
# print("done adding a vector database...")

In [13]:
# # Save to disk
# vector_db.persist()
# print("Vector database created & persisted.")

In [14]:
# Load persisted vector db
vector_db = Chroma(
    persist_directory=persist_dir,
    embedding_function=OllamaEmbeddings(model="nomic-embed-text:v1.5"),
    collection_name="psychology_of_money_rag"
)

print("Loaded existing vector database.")
print("Total items in DB:", vector_db._collection.count())

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13956\3513627814.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_function=OllamaEmbeddings(model="nomic-embed-text:v1.5"),
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13956\3513627814.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(


Loaded existing vector database.
Total items in DB: 412


In [15]:
#setup model
llm = ChatOllama(model = model)

In [16]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [17]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), llm, prompt=QUERY_PROMPT
)

In [18]:
# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [19]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
res = chain.invoke(input=("What is the the book about and give me detailed summary of the book in pointwise manner"))

print(res)